In [1]:
import pandas as pd
import os
import pysam
import re
import collections
from Bio.Seq import Seq

In [2]:
sampleDF = pd.read_csv('/home/mark/Desktop/FunctionalGroup/fullSamples/HG00268_subset.tsv', sep='\t')

In [3]:
sampleDF

,sample,read,contigHap1,locationHap1,contigHap2,locationHap2,flag1,flag2,hap1CIGAR,hap2CIGAR,hap1Seq
0,HG00268,m64119_210712_091919/102171523/ccs,haplotype1-0000001,94567-97865,haplotype2-0000064,77618555-77621853,16,0,2908=6222N88=13870N94=299N206=1S,1S206=299N94=13870N88=6226N2908=,GTAAGATTCCATTTATATGAAGTGTCCAGGGCAGGAAAATCCACAG...
1,HG00268,m64119_210712_091919/93126845/ccs,haplotype1-0000001,106263-108294,haplotype2-0000064,77627773-77629804,0,16,11=1I182=1D85=1D21=1D39=1D110=1D263=2D733N159=...,56=1D35=1D168=1D32=1D439=1D424=1I161=2D733N262...,CACCCCGACGAGGGCGGAGAGCGATGAGGACAACGACGTCGTCATC...
2,HG00268,m64119_210712_091919/132317941/ccs,haplotype1-0000001,116003-118634,haplotype2-0000064,77618176-77620807,16,0,736=1X1893=,1893=1X736=,AAAACTCTATTAAAACTTGCTACATGTGGTGTTAAGGCTTTTTTCT...
3,HG00268,m64119_210712_091919/126157970/ccs,haplotype1-0000001,117047-120001,haplotype2-0000064,77617939-77620893,16,0,1823=1130S,1130S1823=,GTGGATATGAAATTTAATATCCAGAAGAAAAGCACAGAATAAAAAG...
4,HG00268,m64119_210712_091919/3736971/ccs,haplotype1-0000001,125956-129778,haplotype2-0000064,77607034-77610856,16,0,153=1X1466=1X2198=2S,2S2198=1X1466=1X153=,GCAGCGGTGTAACAAATTTTAATTCAAAATAGACACTCTGAAAACA...
...,...,...,...,...,...,...,...,...,...,...,...
995,HG00268,m64119_210712_091919/73730778/ccs,haplotype1-0000001,5779731-5782264,haplotype2-0000064,71933980-71936513,16,0,1S540=1I28=1D553=902N148=309N142=2102N165=2582...,112=1295N144=4431N163=2828N89=1881N56=1D144=14...,GGAGGTGTCATGTTTACTTTTTATTTAGGAGTACAAACTGAGACAA...
996,HG00268,m64119_210712_091919/75434545/ccs,haplotype1-0000001,5779731-5782362,haplotype2-0000064,71930435-71933066,16,0,1S1122=902N148=309N142=2102N165=2582N161=1026N...,2S55=3450N152=1295N144=4431N163=2828N89=1881N2...,GGAGGTGTCATGTTTACTTTTTATTTAGGAGTACAAACTGAGACAA...
997,HG00268,m64119_210712_091919/76154490/ccs,haplotype1-0000001,5779731-5782338,haplotype2-0000064,71933944-71936551,16,0,3S25=1I38=1I5=1I31=1I12=1I8=2I28=1I7=1I2=1I19=...,7S3=1X19=1I125=1295N144=4431N163=2828N89=1881N...,GAGGAGGTGTCATGTTTACTTTTTATTTGAGGAGTACAAACTGAGA...
998,HG00268,m64119_210712_091919/76351524/ccs,haplotype1-0000001,5779731-5782298,haplotype2-0000064,71933948-71936515,16,0,1S1122=902N148=309N142=2102N165=2582N161=1026N...,1S144=1295N144=4431N163=2828N89=1881N201=1434N...,GGAGGTGTCATGTTTACTTTTTATTTAGGAGTACAAACTGAGACAA...


In [18]:
#THIS IS JUST USED TO CHECK THAT THE SEQUENCES ARE CLEAN
#good = ['A', 'C', 'T', 'G']
#for x in sampleDF['hap1Seq']:
#    for y in [a for a in x]:
#        if str(y.upper()) in good:
#            continue
#        else:
#            print(y)

In [4]:
def basicPhaser2(inputCigarString):
    matches = re.findall(r'(\d+)([A-Z=]{1})', inputCigarString)
    tempDict={}
    total=0
    for cig in matches:
        if str(cig[1]) in tempDict.keys():
            tempDict[cig[1]]+=int(cig[0])
        else:
            tempDict[cig[1]]=int(cig[0])
            
    return(tempDict)

In [5]:
h1 = []
h2 = []
for x,y in zip(sampleDF['hap1CIGAR'], sampleDF['hap2CIGAR']):
    h1.append(basicPhaser2(str(x)))
    h2.append(basicPhaser2(str(y)))
sampleDF['BasicPhase_H1']=h1
sampleDF['BasicPhase_H2']=h2

In [6]:
hap1startSites=[]
hap2startSites=[]

for row in sampleDF.index:
    if str(sampleDF.at[row,'hap1CIGAR']) == 'nan' or str(sampleDF.at[row,'hap2CIGAR']) == 'nan':
        hap1startSites.append("PHASEDALREADY")
        hap2startSites.append("PHASEDALREADY")

    else:

        hap1matches = re.findall(r'(\d+)([A-Z=]{1})', str(sampleDF.at[row,'hap1CIGAR']))
        hap2matches = re.findall(r'(\d+)([A-Z=]{1})', str(sampleDF.at[row,'hap2CIGAR']))
        cig1 = hap1matches[0]
        cig2 = hap2matches[0]

        if str(cig1[1]) == 'S':
            hap1startSites.append(int(sampleDF.at[row,'locationHap1'].split("-")[0])-int(cig1[0]))
        else:
            hap1startSites.append(int(sampleDF.at[row,'locationHap1'].split("-")[0]))

        if str(cig2[1]) == 'S':
            hap2startSites.append(int(sampleDF.at[row,'locationHap2'].split("-")[0])-int(cig2[0]))
        else:
            hap2startSites.append(int(sampleDF.at[row,'locationHap2'].split("-")[0]))
sampleDF['Hap1Start']=hap1startSites
sampleDF['Hap2Start']=hap2startSites

In [7]:
def pullSequenceKmer(s, genome, contig, startSite, mywindow,fliporNo):
    
    matches = re.findall(r'(\d+)([A-Z=]{1})', s)
    currentlocation = int(startSite)
    sequences=[]
    
    for cig in matches:
    
        if str(cig[1]) != 'N' and str(cig[1]) != 'S' and str(cig[1]) != 'I':
            goTolocation = currentlocation + (int(cig[0])-1)
            sequences.append(pysam.faidx(genome, str(contig)+":"+str(currentlocation)+"-"+str(goTolocation)).split()[1:])
            currentlocation=goTolocation+1
            
        elif str(cig[1]) == 'I':
            continue
        
        else:
            currentlocation = (currentlocation + (int(cig[0])-1))+1
                
    newSequences = []
    for x in sequences:
        for y in x:
            newSequences.append(y)
           
    mySequence = ''.join(newSequences)
    del newSequences, sequences
    
    if fliporNo.upper() == 'NO':
        mykmers = []
        i =0
        while i < (len(mySequence)-mywindow)+1:
            mykmers.append(mySequence[i:i+mywindow])
            i+=1
    else:
        mySequence2 = Seq(mySequence)
        mySequence3 = str(mySequence2.reverse_complement())
        mykmers = []
        i =0
        while i < (len(mySequence3)-mywindow)+1:
            mykmers.append(mySequence3[i:i+mywindow])
            i+=1
    
    
    return(mykmers)

In [8]:
from tqdm import tqdm
missingHap1 = []
missingHap2 = []
for row in tqdm(sampleDF.index):
    
    if sampleDF.at[row,'Hap1Start'] =='PHASEDALREADY':
        missingHap1.append([])
        missingHap2.append([])
        
    else:
        if str(sampleDF.at[row,'flag1']) == str(sampleDF.at[row,'flag2']):
            fliporNoinput = 'NO'
        else:
            fliporNoinput = 'YES'

        myRead = str(sampleDF.at[row,'hap1Seq'].strip())
        myReadKmers=[]
        window = 15
        i =0
        while i < (len(myRead)-window)+1:
            myReadKmers.append(myRead[i:i+window])
            i+=1


        hap1SeqKmer = pullSequenceKmer(sampleDF.at[row,'hap1CIGAR'],'/home/mark/Desktop/FunctionalGroup/genomes/HG00268.vrk-ps-sseq.asm-hap1.fasta',str(sampleDF.at[row,'contigHap1']),int(sampleDF.at[row,'Hap1Start']),15, 'NO')
        hap2SeqKmer = pullSequenceKmer(sampleDF.at[row,'hap2CIGAR'],'/home/mark/Desktop/FunctionalGroup/genomes/HG00268.vrk-ps-sseq.asm-hap2.fasta',str(sampleDF.at[row,'contigHap2']),int(sampleDF.at[row,'Hap2Start']),15, fliporNoinput)

        mhap1 = []
        mhap2 = []
        for kmer in myReadKmers:
            if kmer in hap1SeqKmer:
                pass
            else:
                mhap1.append(kmer)

            if kmer in hap2SeqKmer:
                pass
            else:
                mhap2.append(kmer)
        missingHap1.append(mhap1)
        missingHap2.append(mhap2)
sampleDF['Hap1Missing']=missingHap1
sampleDF['Hap2Missing']=missingHap2

100%|███████████████████████████████████████| 1000/1000 [01:53<00:00,  8.84it/s]


In [9]:
sampleDF['PhaseV1Results']='temp'
sampleDF['PhaseV2Results']='temp'

for row in sampleDF.index:
    if int(sampleDF.at[row,'BasicPhase_H1']['='])>int(sampleDF.at[row,'BasicPhase_H2']['=']):
        sampleDF.at[row,'PhaseV1Results']='Haplotype1'
    elif int(sampleDF.at[row,'BasicPhase_H1']['='])<int(sampleDF.at[row,'BasicPhase_H2']['=']):
        sampleDF.at[row,'PhaseV1Results']='Haplotype2'
    
    else:
        if sampleDF.at[row,'Hap1Start'] =='PHASEDALREADY':
            if str(sampleDF.at[row,'hap1CIGAR']) == 'nan' and str(sampleDF.at[row,'hap2CIGAR']) != 'nan':
                sampleDF.at[row,'PhaseV1Results']='Haplotype2'
            else:
                sampleDF.at[row,'PhaseV1Results']='Haplotype1'
        else:
            sampleDF.at[row,'PhaseV1Results']='NotAbleToPhase'
        
    if len(sampleDF.at[row,'Hap1Missing'])<len(sampleDF.at[row,'Hap2Missing']):
        sampleDF.at[row,'PhaseV2Results']='Haplotype1'
    elif len(sampleDF.at[row,'Hap1Missing'])>len(sampleDF.at[row,'Hap2Missing']):
        sampleDF.at[row,'PhaseV2Results']='Haplotype2'
    
    else:
        if sampleDF.at[row,'Hap1Start'] =='PHASEDALREADY':
            if str(sampleDF.at[row,'hap1CIGAR']) == 'nan' and str(sampleDF.at[row,'hap2CIGAR']) != 'nan':
                sampleDF.at[row,'PhaseV2Results']='Haplotype2'
            else:
                sampleDF.at[row,'PhaseV2Results']='Haplotype1'
        else:
            sampleDF.at[row,'PhaseV2Results']='NotAbleToPhase'    
    

In [10]:
deepCigar=[]
for row in sampleDF.index:
    total1=0
    total2=0
    
    h1Dict = sampleDF.at[row,'BasicPhase_H1']
    h2Dict = sampleDF.at[row,'BasicPhase_H2']
    
    for key in h1Dict.keys():
        if key == '=':
            total1+=int(h1Dict[key])
        elif key == 'I':
            total1-=int(h1Dict[key])
        elif key == 'S':
            total1-=int(h1Dict[key])
            
        elif key == 'N':
            continue
            
        elif key == 'D':
            total1-=int(h1Dict[key])
            
        elif key == 'X':
            total1-=int(h1Dict[key])
        
        else:
            continue
    for key in h2Dict.keys():
        if key == '=':
            total2+=int(h2Dict[key])
        elif key == 'I':
            total2-=int(h2Dict[key])
        elif key == 'S':
            total2-=int(h2Dict[key])
            
        elif key == 'N':
            continue
            
        elif key == 'D':
            total2-=int(h2Dict[key])
            
        elif key == 'X':
            total2-=int(h2Dict[key])
        
        else:
            continue
            
    if total1 > total2:
        deepCigar.append("Haplotype1")
    elif total2> total1:
        deepCigar.append("Haplotype2")
    else:
        deepCigar.append("NotAbleToPhase")
sampleDF['PhaseV1_DeepResults']=deepCigar            

In [11]:
FinalPhaseList=[]
for row in sampleDF.index:
    tempList =[]
    tempList.append(sampleDF.at[row,'PhaseV1Results'])
    tempList.append(sampleDF.at[row,'PhaseV2Results'])
    tempList.append(sampleDF.at[row,'PhaseV1_DeepResults'])
    #print(tempList)
    tempDict = collections.Counter(tempList)
    #print(tempDict)
    maxPhase = max(tempDict, key=tempDict.get)
    if int(tempDict[maxPhase]) == 1:
        FinalPhaseList.append('NotAbleToPhase')
    else:
        FinalPhaseList.append(maxPhase)
sampleDF['Final_Phase']=FinalPhaseList

In [12]:
sampleDF

,sample,read,contigHap1,locationHap1,contigHap2,locationHap2,flag1,flag2,hap1CIGAR,hap2CIGAR,...,BasicPhase_H1,BasicPhase_H2,Hap1Start,Hap2Start,Hap1Missing,Hap2Missing,PhaseV1Results,PhaseV2Results,PhaseV1_DeepResults,Final_Phase
0,HG00268,m64119_210712_091919/102171523/ccs,haplotype1-0000001,94567-97865,haplotype2-0000064,77618555-77621853,16,0,2908=6222N88=13870N94=299N206=1S,1S206=299N94=13870N88=6226N2908=,...,"{'=': 3296, 'N': 20391, 'S': 1}","{'S': 1, '=': 3296, 'N': 20395}",94567,77618554,[CCAGGGCAGCAGCTC],[CCAGGGCAGCAGCTC],NotAbleToPhase,NotAbleToPhase,NotAbleToPhase,NotAbleToPhase
1,HG00268,m64119_210712_091919/93126845/ccs,haplotype1-0000001,106263-108294,haplotype2-0000064,77627773-77629804,0,16,11=1I182=1D85=1D21=1D39=1D110=1D263=2D733N159=...,56=1D35=1D168=1D32=1D439=1D424=1I161=2D733N262...,...,"{'=': 2028, 'I': 2, 'D': 12, 'N': 733}","{'=': 2028, 'D': 12, 'I': 2, 'N': 733}",106263,77627773,"[CACCCCGACGAGGGC, ACCCCGACGAGGGCG, CCCCGACGAGG...","[CACCCCGACGAGGGC, ACCCCGACGAGGGCG, CCCCGACGAGG...",NotAbleToPhase,NotAbleToPhase,NotAbleToPhase,NotAbleToPhase
2,HG00268,m64119_210712_091919/132317941/ccs,haplotype1-0000001,116003-118634,haplotype2-0000064,77618176-77620807,16,0,736=1X1893=,1893=1X736=,...,"{'=': 2629, 'X': 1}","{'=': 2629, 'X': 1}",116003,77618176,"[CTCTAGAAATATATA, TCTAGAAATATATAT, CTAGAAATATA...","[CTCTAGAAATATATA, TCTAGAAATATATAT, CTAGAAATATA...",NotAbleToPhase,NotAbleToPhase,NotAbleToPhase,NotAbleToPhase
3,HG00268,m64119_210712_091919/126157970/ccs,haplotype1-0000001,117047-120001,haplotype2-0000064,77617939-77620893,16,0,1823=1130S,1130S1823=,...,"{'=': 1823, 'S': 1130}","{'S': 1130, '=': 1823}",117047,77616809,"[CTTCCTGTCGGGCTC, TTCCTGTCGGGCTCT, TCCTGTCGGGC...","[CTTCCTGTCGGGCTC, TTCCTGTCGGGCTCT, TCCTGTCGGGC...",NotAbleToPhase,NotAbleToPhase,NotAbleToPhase,NotAbleToPhase
4,HG00268,m64119_210712_091919/3736971/ccs,haplotype1-0000001,125956-129778,haplotype2-0000064,77607034-77610856,16,0,153=1X1466=1X2198=2S,2S2198=1X1466=1X153=,...,"{'=': 3817, 'X': 2, 'S': 2}","{'S': 2, '=': 3817, 'X': 2}",125956,77607032,"[ATGAACATTATAGCG, TGAACATTATAGCGA, GAACATTATAG...","[ATGAACATTATAGCG, TGAACATTATAGCGA, GAACATTATAG...",NotAbleToPhase,NotAbleToPhase,NotAbleToPhase,NotAbleToPhase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,HG00268,m64119_210712_091919/73730778/ccs,haplotype1-0000001,5779731-5782264,haplotype2-0000064,71933980-71936513,16,0,1S540=1I28=1D553=902N148=309N142=2102N165=2582...,112=1295N144=4431N163=2828N89=1881N56=1D144=14...,...,"{'S': 1, '=': 2529, 'I': 2, 'D': 3, 'N': 18794}","{'=': 2529, 'N': 18621, 'D': 3, 'I': 2, 'S': 1}",5779730,71933980,"[GGAGGTGTCATGTTT, GCTCAAACTCTTGGG, CTCAAACTCTT...","[GGAGGTGTCATGTTT, GCTCAAACTCTTGGG, CTCAAACTCTT...",NotAbleToPhase,NotAbleToPhase,NotAbleToPhase,NotAbleToPhase
996,HG00268,m64119_210712_091919/75434545/ccs,haplotype1-0000001,5779731-5782362,haplotype2-0000064,71930435-71933066,16,0,1S1122=902N148=309N142=2102N165=2582N161=1026N...,2S55=3450N152=1295N144=4431N163=2828N89=1881N2...,...,"{'S': 3, '=': 2625, 'N': 22244, 'X': 2}","{'S': 3, '=': 2627, 'N': 22071}",5779730,71930433,"[GGAGGTGTCATGTTT, CTAGTGGAGACTGAG, TAGTGGAGACT...","[GGAGGTGTCATGTTT, GCAAGCGCGGCGCCC, CAAGCGCGGCG...",Haplotype2,Haplotype2,Haplotype2,Haplotype2
997,HG00268,m64119_210712_091919/76154490/ccs,haplotype1-0000001,5779731-5782338,haplotype2-0000064,71933944-71936551,16,0,3S25=1I38=1I5=1I31=1I12=1I8=2I28=1I7=1I2=1I19=...,7S3=1X19=1I125=1295N144=4431N163=2828N89=1881N...,...,"{'S': 7, '=': 2568, 'I': 31, 'D': 3, 'N': 18794}","{'S': 10, '=': 2564, 'X': 1, 'I': 31, 'N': 186...",5779728,71933937,"[GAGGAGGTGTCATGT, AGGAGGTGTCATGTT, GGAGGTGTCAT...","[GAGGAGGTGTCATGT, AGGAGGTGTCATGTT, GGAGGTGTCAT...",Haplotype1,Haplotype1,Haplotype1,Haplotype1
998,HG00268,m64119_210712_091919/76351524/ccs,haplotype1-0000001,5779731-5782298,haplotype2-0000064,71933948-71936515,16,0,1S1122=902N148=309N142=2102N165=2582N161=1026N...,1S144=1295N144=4431N163=2828N89=1

In [13]:
import collections
print(collections.Counter(sampleDF['PhaseV1Results']))
print(collections.Counter(sampleDF['PhaseV1_DeepResults']))
print(collections.Counter(sampleDF['PhaseV2Results']))
print(collections.Counter(sampleDF['Final_Phase']))

Counter({'Haplotype2': 403, 'Haplotype1': 317, 'NotAbleToPhase': 280})
Counter({'Haplotype1': 417, 'Haplotype2': 404, 'NotAbleToPhase': 179})
Counter({'Haplotype1': 421, 'Haplotype2': 403, 'NotAbleToPhase': 176})
Counter({'Haplotype1': 413, 'Haplotype2': 402, 'NotAbleToPhase': 185})


In [55]:
sampleDF[(sampleDF['PhaseV2Results']=='Haplotype1') & (sampleDF['Final_Phase']=='NotAbleToPhase')]

,sample,read,contigHap1,locationHap1,contigHap2,locationHap2,flag1,flag2,hap1CIGAR,hap2CIGAR,...,BasicPhase_H2,Hap1Start,Hap2Start,Hap1Missing,Hap2Missing,PhaseV1Results,PhaseV2Results,Breakdown,PhaseV1_DeepResults,Final_Phase
46,HG00268,m64119_210712_091919/145165136/ccs,haplotype1-0000001,293599-295689,haplotype2-0000064,77437144-77439234,16,0,1S373=2255N1018=2I553=1825N142=,81=1X60=1825N536=1I1036=2243N373=1S,...,"{'=': 2086, 'X': 1, 'N': 4068, 'I': 1, 'S': 1}",293598,77437144,[GACATTTTTAAGTTT],"[GACATTTTTAAGTTT, TCCATGTGGACCGCG, CCATGTGGACC...",NotAbleToPhase,Haplotype1,noPerfectMatch,Haplotype2,NotAbleToPhase
464,HG00268,m64119_210712_091919/29230379/ccs,haplotype1-0000001,3166242-3171367,haplotype2-0000064,74450787-74455912,0,16,479=26221N76=39010N88=48869N349=1I494=1I260=1I...,436=1I525=1I493=1I1017=1I888=1I266=1I497=1I352...,...,"{'=': 5116, 'I': 7, 'N': 114023, 'X': 1}",3166242,74450787,"[CATGGCTTCCTCCCC, ATGGCTTCCTCCCCA, TGGCTTCCTCC...","[CCAGGAGAGCCCGGA, CAGGAGAGCCCGGAT, AGGAGAGCCCG...",NotAbleToPhase,Haplotype1,noPerfectMatch,Haplotype2,NotAbleToPhase
568,HG00268,m64119_210712_091919/64226014/ccs,haplotype1-0000001,5090954-5094461,haplotype2-0000064,72635129-72638636,0,16,1S310=6509N1100=1I2094=,2095=1I1099=6509N310=1S,...,"{'=': 3504, 'I': 1, 'N': 6509, 'S': 1}",5090953,72635129,"[GCCCGCCCGCCCTAG, GGGGAAAAAAAAAAA, GGGAAAAAAAA...","[GCCCGCCCGCCCTAG, TCTTCCCTTGGGGGG, CTTCCCTTGGG...",NotAbleToPhase,Haplotype1,noPerfectMatch,NotAbleToPhase,NotAbleToPhase
569,HG00268,m64119_210712_091919/120588203/ccs,haplotype1-0000001,5090974-5093553,haplotype2-0000064,72636037-72638616,0,16,1S290=6509N1100=1I1186=,1187=1I1099=6509N290=1S,...,"{'=': 2576, 'I': 1, 'N': 6509, 'S': 1}",5090973,72636037,"[GAGAGCACAGCCGAG, GGGGAAAAAAAAAAA, GGGAAAAAAAA...","[GAGAGCACAGCCGAG, TCTTCCCTTGGGGGG, CTTCCCTTGGG...",NotAbleToPhase,Haplotype1,noPerfectMatch,NotAbleToPhase,NotAbleToPhase
573,HG00268,m64119_210712_091919/126222455/ccs,haplotype1-0000001,5090979-5095855,haplotype2-0000064,72633734-72638610,0,16,285=6509N7=1D1092=1I3489=1S,1S3490=1I1090=1D8=6509N285=,...,"{'S': 1, '=': 4873, 'I': 1, 'D': 1, 'N': 6509}",5090979,72633733,"[GAACCGATTTGTTGT, AACCGATTTGTTGTG, ACCGATTTGTT...","[GAACCGATTTGTTGT, AACCGATTTGTTGTG, ACCGATTTGTT...",NotAbleToPhase,Haplotype1,noPerfectMatch,NotAbleToPhase,NotAbleToPhase
580,HG00268,m64119_210712_091919/26346562/ccs,haplotype1-0000001,5091010-5094148,haplotype2-0000064,72635447-72638585,0,16,2S132=1I122=6509N337=1I530=1I233=1I364=1I1412=,1409=1I368=1I226=1I531=1I342=6509N118=1I136=2S,...,"{'=': 3130, 'I': 5, 'N': 6509, 'S': 2}",5091008,72635447,"[GGGACCCCGGCCAGC, GGACCCCGGCCAGCG, CTCCGCCATTC...","[GGGACCCCGGCCAGC, GGACCCCGGCCAGCG, CTCCGCCATTC...",NotAbleToPhase,Haplotype1,noPerfectMatch,NotAbleToPhase,NotAbleToPhase
588,HG00268,m64119_210712_091919/24382274/ccs,haplotype1-0000001,5091020-5095073,haplotype2-0000064,72634516-72638569,0,16,244=6509N871=1X228=1I2707=,2708=1I227=1X871=6509N244=,...,"{'=': 4050, 'I': 1, 'X': 1, 'N': 6509}",5091020,72634516,"[TGTACATGGGAAAAG, GTACATGGGAAAAGA, TACATGGGAAA...","[TGTACATGGGAAAAG, GTACATGGGAAAAGA, TACATGGGAAA...",NotAbleToPhase,Haplotype1,noPerfectMatch,NotAbleToPhase,NotAbleToPhase
591,HG00268,m64119_210712_091919/71894259/ccs,haplotype1-0000001,5091586-5093996,haplotype2-0000064,72636038-72638448,0,16,1S108=6079N310=1I42=1I48=1I654=1X30=1X14=1I42=...,1S34=1I28=1I441=1I2=1I65=1I37=1I23=1I337=1I76=...,...,"{'S': 2, '=': 2391, 'I': 14, 'X': 2, 'N': 6079}",5091585,72636037,"[GAGAAGGGGAGGCTC, CGGAAGGGAAAAAAA, GGAAGGGAAAA...","[GAGAAGGGGAGGCTC, CGGAAGGGAAAAAAA, GGAAGGGAAAA...",NotAbleToPhase,Haplotype1,noPerfectMatch,NotAbleToPhase,NotAbleToPhase
593,HG00268,m64119_210712_091919/87031924/ccs,haplotype1-0000001,5097771-5102850,haplotype2-0000064,72633735-72638814,0,16,487S1102=1I649=1D1044=1I1794=,1788=1I1050=1D650=1I1101=487S,...,"{'=': 4589, 'I': 2, 'D': 1, 'S': 487}",5097284,72633735,"[GGACTGCCCAGCTGG, GACTGCCCAGCTGGG, ACTGCCCAGCT...","[GGACTGCCCAGCTGG, GACTGCCCAGCTGGG, ACTGCCCAG

In [51]:
for x in sampleDF[(sampleDF['PhaseV2Results']=='Haplotype2') & (sampleDF['Final_Phase']=='NotAbleToPhase')]['Hap2Missing']:
    print(len(x))

for x in sampleDF[(sampleDF['PhaseV2Results']=='Haplotype2') & (sampleDF['Final_Phase']=='NotAbleToPhase')]['Hap1Missing']:
    print(len(x))

154
159


In [47]:
sampleDF[(sampleDF['PhaseV2Results']=='Haplotype2') & (sampleDF['Final_Phase']=='NotAbleToPhase')]

,sample,read,contigHap1,locationHap1,contigHap2,locationHap2,flag1,flag2,hap1CIGAR,hap2CIGAR,...,BasicPhase_H2,Hap1Start,Hap2Start,Hap1Missing,Hap2Missing,PhaseV1Results,PhaseV2Results,Breakdown,PhaseV1_DeepResults,Final_Phase
592,HG00268,m64119_210712_091919/18155590/ccs,haplotype1-0000001,5097771-5101120,haplotype2-0000064,72635111-72638460,0,16,135S229=1X866=1D2117=,2102=1D881=1X229=135S,...,"{'=': 3212, 'D': 1, 'X': 1, 'S': 135}",5097636,72635111,"[GGAGACGCGACCAGG, GAGACGCGACCAGGA, AGACGCGACCA...","[GGAGACGCGACCAGG, GAGACGCGACCAGGA, AGACGCGACCA...",NotAbleToPhase,Haplotype2,noPerfectMatch,NotAbleToPhase,NotAbleToPhase


In [65]:
tempbreak = []
for row in sampleDF.index:
    perfectone=0
    nonperfectone=0
    
    perfecttwo=0
    nonperfecttwo=0

    if len(sampleDF.at[row,'BasicPhase_H1'].keys()) == 1 or  len(sampleDF.at[row,'BasicPhase_H1'].keys()) == 2:
        for y in [x for x in sampleDF.at[0,'BasicPhase_H1'].keys()]:
            if y == '=' or y == 'N':
                perfectone+=1
            else:
                nonperfectone+=1
    else:
        nonperfectone+=1
     
    if len(sampleDF.at[row,'BasicPhase_H2'].keys()) == 1 or  len(sampleDF.at[row,'BasicPhase_H2'].keys()) == 2:
        for y in [x for x in sampleDF.at[0,'BasicPhase_H2'].keys()]:
            if y == '=' or y == 'N':
                perfecttwo+=1
            else:
                nonperfecttwo+=1
    else:
        nonperfecttwo+=1
        
    if (perfectone >0 and nonperfectone==0) and (perfecttwo >0 and nonperfecttwo==0):
        tempbreak.append('BothPerfectMatches')
    elif (perfectone > 0 and nonperfectone==0 and nonperfecttwo >0) or (perfecttwo > 0  and nonperfecttwo==0 and nonperfectone >0):
        tempbreak.append('onePerfectMatch')
    else:
        tempbreak.append('noPerfectMatch')
sampleDF['Breakdown'] =tempbreak 

In [66]:
print(collections.Counter(sampleDF['Breakdown']))

Counter({'noPerfectMatch': 1000})


In [64]:
sampleDF[sampleDF['Breakdown']=='onePerfectMatch']

,sample,read,contigHap1,locationHap1,contigHap2,locationHap2,flag1,flag2,hap1CIGAR,hap2CIGAR,...,BasicPhase_H2,Hap1Start,Hap2Start,Hap1Missing,Hap2Missing,PhaseV1Results,PhaseV2Results,Breakdown,PhaseV1_DeepResults,Final_Phase
2,HG00268,m64119_210712_091919/132317941/ccs,haplotype1-0000001,116003-118634,haplotype2-0000064,77618176-77620807,16,0,736=1X1893=,1893=1X736=,...,"{'=': 2629, 'X': 1}",116003,77618176,"[CTCTAGAAATATATA, TCTAGAAATATATAT, CTAGAAATATA...","[CTCTAGAAATATATA, TCTAGAAATATATAT, CTAGAAATATA...",NotAbleToPhase,NotAbleToPhase,onePerfectMatch,NotAbleToPhase,NotAbleToPhase
3,HG00268,m64119_210712_091919/126157970/ccs,haplotype1-0000001,117047-120001,haplotype2-0000064,77617939-77620893,16,0,1823=1130S,1130S1823=,...,"{'S': 1130, '=': 1823}",117047,77616809,"[CTTCCTGTCGGGCTC, TTCCTGTCGGGCTCT, TCCTGTCGGGC...","[CTTCCTGTCGGGCTC, TTCCTGTCGGGCTCT, TCCTGTCGGGC...",NotAbleToPhase,NotAbleToPhase,onePerfectMatch,NotAbleToPhase,NotAbleToPhase
8,HG00268,m64119_210712_091919/29100612/ccs,haplotype1-0000001,127485-130280,haplotype2-0000064,77606527-77609322,16,0,2190=1D14=1D129=1D461=,459=1D128=1D15=1D2192=,...,"{'=': 2794, 'D': 3}",127485,77606527,"[GTCCCACAGGCTGGT, TCCCACAGGCTGGTG, CCCACAGGCTG...","[GTCCCACAGGCTGGT, TCCCACAGGCTGGTG, CCCACAGGCTG...",NotAbleToPhase,NotAbleToPhase,onePerfectMatch,NotAbleToPhase,NotAbleToPhase
9,HG00268,m64119_210712_091919/76744772/ccs,haplotype1-0000001,127485-129330,haplotype2-0000064,77607469-77609314,16,0,356=1D29=2D1132=2D24=2D44=1D40=1D27=1D56=1D136=,135=1D56=1D27=1D39=1D43=2D24=2D1133=2D30=1D357=,...,"{'=': 1844, 'D': 11}",127485,77607469,"[TTGTCTTTTAAAAGC, TGTCTTTTAAAAGCA, GTCTTTTAAAA...","[TTGTCTTTTAAAAGC, TGTCTTTTAAAAGCA, GTCTTTTAAAA...",NotAbleToPhase,NotAbleToPhase,onePerfectMatch,NotAbleToPhase,NotAbleToPhase
10,HG00268,m64119_210712_091919/107544667/ccs,haplotype1-0000001,127485-130948,haplotype2-0000064,77579955-77583418,16,0,3205=2419N90=15431N131=8055N36=,36=8055N131=15431N90=2421N3205=,...,"{'=': 3462, 'N': 25907}",127485,77579955,[],[],NotAbleToPhase,NotAbleToPhase,onePerfectMatch,NotAbleToPhase,NotAbleToPhase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,HG00268,m64119_210712_091919/173212438/ccs,haplotype1-0000001,5779730-5782362,haplotype2-0000064,71930432-71933064,16,0,1123=902N148=309N142=2102N165=2582N161=1026N85...,58=3450N152=1295N144=4431N163=2828N89=1881N201...,...,"{'=': 2631, 'N': 22071}",5779730,71930432,"[CTAGTGGAGACTGAG, TAGTGGAGACTGAGG, AGTGGAGACTG...",[],Haplotype2,Haplotype2,onePerfectMatch,Haplotype2,Haplotype2
916,HG00268,m64119_210712_091919/173475725/ccs,haplotype1-0000001,5779730-5782389,haplotype2-0000064,71930405-71933064,16,0,1123=902N148=309N142=2102N165=2582N161=1026N85...,10=1X74=3450N3=1X3=1X144=1295N144=4431N163=282...,...,"{'=': 2655, 'X': 3, 'N': 22071}",5779730,71930405,[],"[CTAGTGGAGACTGAC, TAGTGGAGACTGACG, AGTGGAGACTG...",Haplotype1,Haplotype1,onePerfectMatch,Haplotype1,Haplotype1
920,HG00268,m64119_210712_091919/174981513/ccs,haplotype1-0000001,5779730-5782309,haplotype2-0000064,71933935-71936514,16,0,1123=902N148=309N142=2102N165=2582N161=1026N85...,157=1295N144=4431N163=2828N89=1881N201=1434N85...,...,"{'=': 2578, 'N': 18621}",5779730,71933935,"[CTAGTGGAGACTGAG, TAGTGGAGACTGAGG, AGTGGAGACTG...",[],Haplotype2,Haplotype2,onePerfectMatch,Haplotype2,Haplotype2
925,HG00268,m64119_210712_091919/178259518/ccs,haplotype1-0000001,5779730-5782309,haplotype2-0000064,71933935-71936514,16,0,1123=902N148=309N142=2102N165=2582N161=1026N85...,8=1X3=1X144=1295N144=4431N163=2828N89=1881N201...,...,"{'=': 2576, 'X': 2, 'N': 18621}",5779730,71933935,[],"[CTAGTGGAGACTGAC, TAGTGGAGACTGACG, AGTGGAGACTG...",Haplotype1,Haplotype1,onePerfectMatch,Haplotype1,Haplotype1
